<a href="https://colab.research.google.com/github/Jonathan-code-hub/Many-Mini-OR-Problems/blob/main/Combinatorics%2BLP/Combinatoric_LP_Problem_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Problem 1: Combinatorics & Linear Programming

You are helping to plan seating arrangements for a small event. There are **5 tables** and **20 guests**.

---

### Part A: Combinatorics
1. Each table can seat exactly 4 guests. How many different ways can the 20 guests be partitioned into the 5 tables (ignoring the order of tables and the order of guests at each table)?
2. Suppose two guests, Alice and Bob, insist on sitting at the same table. How does this restriction change the total number of possible partitions?

---

### Part B: Linear Programming
The event host wants to assign guests to tables while **minimizing conflicts**. Define a *conflict pair* as two guests who dislike each other. The event planner has a list of conflict pairs among the 20 guests.

1. Define binary decision variables  
   \[
   x_{i,j} =
   \begin{cases}
   1 & \text{if guest $i$ sits at table $j$}, \\
   0 & \text{otherwise}.
   \end{cases}
   \]

2. Write the **constraints** to ensure:
   - Every guest is assigned to exactly one table.
   - Each table seats exactly 4 guests.

3. Formulate an **objective function** to minimize the total number of conflict pairs seated at the same table.

---

### Bonus Challenge
If Alice and Bob must sit together, how would you **modify the constraints** to reflect this condition?

---


In [ ]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 89.0 MB/s eta 0:00:00


In [ ]:
import pulp

# Problem setup #
num_guests = 20
num_tables = 5
seats_per_table = 4

guests = list(range(num_guests))
tables = list(range(num_tables))

# Example conflict pairs (modify as needed) #
conflict_pairs = [(0, 1), (2, 3), (4, 5), (6, 7), (1, 2), (8, 9)]

# Define LP problem #
prob = pulp.LpProblem("Guest_Seating", pulp.LpMinimize)

# Binary decision variables: x[i][j] = 1 if guest i sits at table j #
x = pulp.LpVariable.dicts("x", (guests, tables), cat="Binary")

# Conflict helper variables: y[i,k,j] = 1 if guests i and k both sit at table j #
y = pulp.LpVariable.dicts("y", (range(len(conflict_pairs)), tables), cat="Binary")

# Constraints #

# Each guest must sit at exactly one table #
for i in guests:
    prob += pulp.lpSum(x[i][j] for j in tables) == 1

# Each table must have exactly seats_per_table guests #
for j in tables:
    prob += pulp.lpSum(x[i][j] for i in guests) == seats_per_table

# Define y[i,k,j] in terms of x[i][j] and x[k][j] #
for idx, (i, k) in enumerate(conflict_pairs):
    for j in tables:
        prob += y[idx][j] <= x[i][j]
        prob += y[idx][j] <= x[k][j]
        prob += y[idx][j] >= x[i][j] + x[k][j] - 1

# Objective function #
# Minimize total number of conflicts (sum of all y[i,k,j]) #
prob += pulp.lpSum(y[idx][j] for idx in range(len(conflict_pairs)) for j in tables)

# Solve the problem #
prob.solve(pulp.PULP_CBC_CMD(msg=0))

print("Status:", pulp.LpStatus[prob.status])
print("Total conflicts:", pulp.value(prob.objective))

# Print seating plan #
for j in tables:
    seated = [i for i in guests if pulp.value(x[i][j]) == 1]
    print(f"Table {j}: {seated}")


Status: Optimal
Total conflicts: 0.0
Table 0: [2, 5, 14, 15]
Table 1: [0, 8, 10, 12]
Table 2: [7, 9, 11, 17]
Table 3: [3, 13, 16, 18]
Table 4: [1, 4, 6, 19]
